In [1]:
# @title 書割効果シミュレーター
import numpy as np
import cv2
import ipywidgets as widgets
from IPython.display import display, clear_output
from google.colab import files

# ==========================================
# 1. レンダリング・エンジン
# ==========================================

def render_scene_final_full(h, w, visual_radius_px, sphere_dist_m, eye_baseline_m, shaping_factor, is_fixed_size, grid_alpha, tex_mode='geo'):
    # --- カメラ設定 ---
    pixel_pitch_mm = 0.3
    screen_dist_mm = 500.0
    f_px = screen_dist_mm / pixel_pitch_mm

    # --- サイズ計算 ---
    if is_fixed_size:
        real_radius_m = 2.0
    else:
        half_fov_rad = np.arctan((visual_radius_px * pixel_pitch_mm) / screen_dist_mm)
        real_radius_m = sphere_dist_m * np.tan(half_fov_rad)

    # --- 座標定義 ---
    pole_height = 1.6
    camera_height = 1.6

    # 球体中心
    sphere_center_y = pole_height + real_radius_m
    sphere_center = np.array([0.0, sphere_center_y, sphere_dist_m])

    # 支柱の半径
    pole_radius = max(0.05, real_radius_m * 0.1)

    # 建物
    wall_x_L = -6.0; wall_x_R = 6.0; wall_height = 20.0

    eye_offset_m = eye_baseline_m / 2.0

    X_1d = np.arange(w); Y_1d = np.arange(h)
    X, Y = np.meshgrid(X_1d, Y_1d)
    cx, cy = w // 2, h // 2

    def render_view(current_eye_offset):
        # 視点 P
        Px = current_eye_offset
        Py = camera_height
        Pz = 0.0

        # レイ V
        vx = (X - cx) / f_px
        vy = -(Y - cy) / f_px
        vz = np.ones_like(X, dtype=np.float32)
        norm = np.sqrt(vx**2 + vy**2 + vz**2)
        vx /= norm; vy /= norm; vz /= norm

        t_final = np.full((h, w), np.inf, dtype=np.float32)
        obj_id = np.zeros((h, w), dtype=int)

        # =========================================================
        # ★視線軸変形 (View-Axis Shaping)
        # =========================================================
        cam_center = np.array([0.0, camera_height, 0.0])
        axis_vec = sphere_center - cam_center
        axis_len = np.linalg.norm(axis_vec)
        if axis_len < 1e-6: axis_vec = np.array([0,0,1])
        else: axis_vec /= axis_len

        fwd = axis_vec
        world_up = np.array([0.0, 1.0, 0.0])
        if np.abs(np.dot(fwd, world_up)) > 0.99:
            right = np.array([1.0, 0.0, 0.0])
        else:
            right = np.cross(world_up, fwd)
            right /= np.linalg.norm(right)
        up = np.cross(fwd, right)
        up /= np.linalg.norm(up)

        R_mat = np.stack([right, up, fwd])

        Lx_w = Px - sphere_center[0]
        Ly_w = Py - sphere_center[1]
        Lz_w = Pz - sphere_center[2]

        P_loc_x = R_mat[0,0]*Lx_w + R_mat[0,1]*Ly_w + R_mat[0,2]*Lz_w
        P_loc_y = R_mat[1,0]*Lx_w + R_mat[1,1]*Ly_w + R_mat[1,2]*Lz_w
        P_loc_z = R_mat[2,0]*Lx_w + R_mat[2,1]*Ly_w + R_mat[2,2]*Lz_w

        V_loc_x = R_mat[0,0]*vx + R_mat[0,1]*vy + R_mat[0,2]*vz
        V_loc_y = R_mat[1,0]*vx + R_mat[1,1]*vy + R_mat[1,2]*vz
        V_loc_z = R_mat[2,0]*vx + R_mat[2,1]*vy + R_mat[2,2]*vz

        scale_inv = 1.0 / shaping_factor
        Dx = V_loc_x; Dy = V_loc_y; Dz = V_loc_z * scale_inv
        Ox = P_loc_x; Oy = P_loc_y; Oz = P_loc_z * scale_inv

        a = Dx**2 + Dy**2 + Dz**2
        b = 2 * (Dx * Ox + Dy * Oy + Dz * Oz)
        c = Ox**2 + Oy**2 + Oz**2 - real_radius_m**2

        disc = b**2 - 4 * a * c
        hit_sphere = disc >= 0

        if np.any(hit_sphere):
            t_calc = (-b[hit_sphere] - np.sqrt(disc[hit_sphere])) / (2 * a[hit_sphere])
            valid_s = t_calc > 0

            indices = np.where(hit_sphere)
            current_z = t_final[hit_sphere]
            update = valid_s & (t_calc < current_z)

            y_idx = indices[0][update]; x_idx = indices[1][update]
            t_final[y_idx, x_idx] = t_calc[update]
            obj_id[y_idx, x_idx] = 1

        # 2. 支柱 (Pole)
        Lx_p = Px - 0; Lz_p = Pz - sphere_dist_m
        a_p = vx**2 + vz**2
        b_p = 2 * (Lx_p * vx + Lz_p * vz)
        c_p = Lx_p**2 + Lz_p**2 - pole_radius**2

        disc_p = b_p**2 - 4 * a_p * c_p
        hit_pole = disc_p >= 0

        if np.any(hit_pole):
            t_p = (-b_p[hit_pole] - np.sqrt(disc_p[hit_pole])) / (2 * a_p[hit_pole])
            hit_y = Py + t_p * vy[hit_pole]
            valid_p = (t_p > 0) & (hit_y >= 0) & (hit_y <= pole_height)

            wm = np.zeros((h,w), bool)
            idx = np.where(hit_pole)
            wm[idx[0][valid_p], idx[1][valid_p]] = True
            mask_z = t_p[valid_p] < t_final[wm]

            y_i, x_i = np.where(wm)
            t_final[y_i[mask_z], x_i[mask_z]] = t_p[valid_p][mask_z]
            obj_id[y_i[mask_z], x_i[mask_z]] = 5

        # 3. 地面
        valid_g = vy < -1e-4
        if np.any(valid_g):
            t_g = -Py / vy[valid_g]
            valid_t = t_g > 0
            wm = np.zeros((h,w), bool); idx = np.where(valid_g)
            wm[idx[0][valid_t], idx[1][valid_t]] = True
            mask_z = t_g[valid_t] < t_final[wm]
            y_i, x_i = np.where(wm)
            t_final[y_i[mask_z], x_i[mask_z]] = t_g[valid_t][mask_z]
            obj_id[y_i[mask_z], x_i[mask_z]] = 2

        # 4. 壁
        for wall_x, oid in [(wall_x_L, 3), (wall_x_R, 4)]:
            vx_s = vx.copy(); vx_s[np.abs(vx_s)<1e-5] = 1e-5
            t_w = (wall_x - Px) / vx_s
            hit_y = Py + t_w * vy
            valid_w = (t_w > 0) & (hit_y >= 0) & (hit_y <= wall_height)
            if np.any(valid_w):
                wm = np.zeros((h,w), bool); idx = np.where(valid_w)
                wm[idx[0], idx[1]] = True
                mask_z = t_w[valid_w] < t_final[wm]
                y_i, x_i = np.where(wm)
                t_final[y_i[mask_z], x_i[mask_z]] = t_w[valid_w][mask_z]
                obj_id[y_i[mask_z], x_i[mask_z]] = oid

        # --- シェーディング ---
        sky = np.clip(0.5 - (Y/h), 0, 1)*2.0
        b_img = 255*(0.6+0.4*sky); g_img = 255*(0.7+0.2*sky); r_img = 255*(0.9+0.1*sky)

        # 球体
        mask = (obj_id == 1)
        if np.any(mask):
            ts = t_final[mask]
            Ox_m = P_loc_x; Oy_m = P_loc_y; Oz_m = P_loc_z
            Vx_m = V_loc_x[mask]; Vy_m = V_loc_y[mask]; Vz_m = V_loc_z[mask]
            Qx_loc = Ox_m + ts * Vx_m
            Qy_loc = Oy_m + ts * Vy_m
            Qz_loc = Oz_m + ts * Vz_m

            norm_Qx = Qx_loc / real_radius_m
            norm_Qy = Qy_loc / real_radius_m
            norm_Qz = (Qz_loc * scale_inv) / real_radius_m

            Nx_w = R_mat[0,0]*norm_Qx + R_mat[1,0]*norm_Qy + R_mat[2,0]*norm_Qz
            Ny_w = R_mat[0,1]*norm_Qx + R_mat[1,1]*norm_Qy + R_mat[2,1]*norm_Qz
            Nz_w = R_mat[0,2]*norm_Qx + R_mat[1,2]*norm_Qy + R_mat[2,2]*norm_Qz

            if tex_mode == 'geo':
                lat = np.degrees(np.arcsin(np.clip(Ny_w, -1, 1)))
                lon = np.degrees(np.arctan2(Nx_w, -Nz_w))
                step = 15
                is_grid = (np.abs(lat.astype(int)) % step < 2) | (np.abs(lon.astype(int)) % step < 2)
                is_center = (np.abs(lat) < 3) & (np.abs(lon) < 3)
                is_axis = (np.abs(lat) < 1.5) | (np.abs(lon) < 1.5)
            else:
                u = np.arcsin(np.clip(Nx_w, -1, 1)) * 180 / np.pi
                v = np.arcsin(np.clip(Ny_w, -1, 1)) * 180 / np.pi
                step = 15
                is_grid = (np.abs(u.astype(int)) % step < 2) | (np.abs(v.astype(int)) % step < 2)
                is_center = (np.sqrt(u**2 + v**2) < 6)
                is_axis = (np.abs(u) < 2) | (np.abs(v) < 2)

            L_len = np.sqrt(Nx_w**2+Ny_w**2+Nz_w**2)
            diff = np.maximum(0, (Nx_w*0.5 + Ny_w*0.6 + Nz_w*(-0.6))/L_len) * 100

            base = 100 + diff
            b_s = base.copy(); g_s = base.copy(); r_s = base.copy()

            # グリッドのアルファ合成
            # Grid Color: 240 (白に近い)
            # Alpha blending: Final = Grid * Alpha + Base * (1 - Alpha)

            grid_val = 240.0

            # グリッド
            b_s[is_grid] = grid_val * grid_alpha + b_s[is_grid] * (1 - grid_alpha)
            g_s[is_grid] = grid_val * grid_alpha + g_s[is_grid] * (1 - grid_alpha)
            r_s[is_grid] = grid_val * grid_alpha + r_s[is_grid] * (1 - grid_alpha)

            # 軸と中心点 (これらは重要なマーカーなのでAlphaの影響を少し弱めるか、同様にするか)
            # ここでは同様にAlphaを適用します
            marker_val_b = 0.0; marker_val_g = 255.0; marker_val_r = 0.0 # 緑
            center_val_b = 0.0; center_val_g = 0.0;   center_val_r = 255.0 # 赤

            b_s[is_axis] = marker_val_b * grid_alpha + b_s[is_axis] * (1 - grid_alpha)
            g_s[is_axis] = marker_val_g * grid_alpha + g_s[is_axis] * (1 - grid_alpha)
            r_s[is_axis] = marker_val_r * grid_alpha + r_s[is_axis] * (1 - grid_alpha)

            b_s[is_center] = center_val_b * grid_alpha + b_s[is_center] * (1 - grid_alpha)
            g_s[is_center] = center_val_g * grid_alpha + g_s[is_center] * (1 - grid_alpha)
            r_s[is_center] = center_val_r * grid_alpha + r_s[is_center] * (1 - grid_alpha)

            b_img[mask] = b_s; g_img[mask] = g_s; r_img[mask] = r_s

        # 柱・地面・壁
        mask = (obj_id == 5)
        if np.any(mask):
            ts = t_final[mask]
            Qx = Px + ts * vx[mask]; Qz = Pz + ts * vz[mask]
            Nx = Qx/pole_radius; Nz = (Qz-sphere_dist_m)/pole_radius
            d = np.clip(Nx*0.5 + Nz*(-0.8), 0.2, 1.0)*100 + 80
            b_img[mask]=d; g_img[mask]=d; r_img[mask]=d

        mask = (obj_id == 2)
        if np.any(mask):
            ts = t_final[mask]
            Qx = Px + ts * vx[mask]; Qz = Pz + ts * vz[mask]
            lw = 0.15; dl = 4.0
            ic = np.abs(Qx)<lw; id = (Qz%dl)<(dl*0.6)
            c = np.where(ic&id, 220., 60.)
            g = ((np.floor(Qx)+np.floor(Qz))%2==0)
            c = np.where((~(ic&id))&g, 70., c)
            f = np.exp(-ts*0.01)
            c = c*f + 200*(1-f)
            b_img[mask]=c; g_img[mask]=c; r_img[mask]=c

        mask = (obj_id == 3)|(obj_id == 4)
        if np.any(mask):
            ts = t_final[mask]
            Qy = Py + ts * vy[mask]; Qz = Pz + ts * vz[mask]
            iw = ((np.floor(Qy/3)%2==0)&(np.floor(Qz/5)%2==0))
            c = np.where(iw, 50., 160.)
            iL = (obj_id[mask]==3)
            bw=c.copy(); gw=c.copy(); rw=c.copy()
            bw[iL]+=50; rw[~iL]+=50
            f = np.exp(-ts*0.01)
            bw = bw*f + 200*(1-f); gw = gw*f + 200*(1-f); rw = rw*f + 200*(1-f)
            b_img[mask]=bw; g_img[mask]=gw; r_img[mask]=rw

        return cv2.merge([b_img, g_img, r_img]).astype(np.uint8)

    img_L = render_view(-eye_offset_m)
    img_R = render_view(eye_offset_m)
    return img_L, img_R, real_radius_m

# --- UI ---
style = {'description_width': 'initial'}
layout_full = widgets.Layout(width='95%')
current_img = None

slider_size = widgets.IntSlider(value=800, min=200, max=1500, step=50, description='画面サイズ (px):', style=style, layout=layout_full)
slider_dist = widgets.FloatLogSlider(value=10.0, base=10, min=0.0, max=2.5, step=0.1, description='球体の距離 (m):', style=style, layout=layout_full)
slider_shape = widgets.FloatSlider(value=1.0, min=1.0, max=20.0, step=1.0, description='整形倍率 (Shaping):', style=style, layout=layout_full)
slider_eye = widgets.FloatSlider(value=0.065, min=0.01, max=0.2, step=0.005, description='目の幅 (m):', style=style, layout=layout_full)
slider_grid_alpha = widgets.FloatSlider(value=0.2, min=0.0, max=1.0, step=0.1, description='線の濃さ (Alpha):', style=style, layout=layout_full)

checkbox_fixed_size = widgets.Checkbox(value=True, description='物理サイズを固定 (Fixed Real Size)')

dd_mode = widgets.Dropdown(options=['Cross-Eyed (交差法)', 'Parallel (平行法)'], value='Parallel (平行法)', description='観察モード:', style=style)
dd_tex = widgets.Dropdown(options=['Meridians (子午線)', 'Orthographic (直交)'], value='Orthographic (直交)', description='テクスチャ:', style=style)
btn_save = widgets.Button(description='画像を保存', icon='download', button_style='success')
out = widgets.Output()

def update(change=None):
    global current_img
    dist = slider_dist.value
    eye = slider_eye.value
    shape = slider_shape.value
    mode = dd_mode.value
    tex = 'geo' if 'Meridians' in dd_tex.value else 'ortho'
    size = slider_size.value
    fixed_size = checkbox_fixed_size.value
    grid_alpha = slider_grid_alpha.value

    visual_rad_base = 120

    img_L, img_R, rr = render_scene_final_full(400, 600, visual_rad_base, dist, eye, shape, fixed_size, grid_alpha, tex)

    def decorate(img, t):
        o = img.copy()
        h, w = o.shape[:2]
        cx, cy = w//2, h//2
        cv2.line(o, (cx-15, cy), (cx+15, cy), (0,255,255), 2)
        cv2.line(o, (cx, cy-15), (cx, cy+15), (0,255,255), 2)
        cv2.rectangle(o, (0,0), (w-1,h-1), (255,255,255), 2)
        cv2.putText(o, t, (20,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
        return o

    if mode == 'Parallel (平行法)':
        comb = np.hstack((decorate(img_L,"L"), decorate(img_R,"R")))
    else:
        comb = np.hstack((decorate(img_R,"R"), decorate(img_L,"L")))

    current_img = comb
    _, enc = cv2.imencode('.png', comb)

    out.clear_output(wait=True)
    with out:
        display(widgets.Image(value=enc.tobytes(), format='png', width=f"{size}px"))
        print(f"【物理パラメータ】")
        print(f"・距離: {dist:.1f} m")
        print(f"・線の濃さ: {grid_alpha:.1f}")
        print(f"・整形倍率: {shape}倍")

def save_img(b):
    if current_img is not None:
        fn = f"stereo_final_full_dist{slider_dist.value}_shape{slider_shape.value}.png"
        cv2.imwrite(fn, current_img)
        files.download(fn)

slider_size.observe(update, names='value')
slider_dist.observe(update, names='value')
slider_shape.observe(update, names='value')
slider_eye.observe(update, names='value')
slider_grid_alpha.observe(update, names='value')
checkbox_fixed_size.observe(update, names='value')
dd_mode.observe(update, names='value')
dd_tex.observe(update, names='value')
btn_save.on_click(save_img)

update()

display(widgets.VBox([
    widgets.HBox([dd_mode, dd_tex, checkbox_fixed_size, btn_save]),
    slider_size,
    slider_dist,
    slider_shape,
    slider_grid_alpha,
    slider_eye,
    out
]))